In [ ]:
from datawand.parametrization import ConfigGenerator

# Initialize configuration generator

In [ ]:
cg = ConfigGenerator(pipeline_name='LNGraph')

# Setting default parameters for pipeline

In [ ]:
DEFAULTS = {}

#### Set your default parameters below:

In [ ]:
DEFAULTS["data_dir"] = "/mnt/idms/fberes/data/bitcoin_ln_research"
DEFAULTS["weight"] = None
DEFAULTS["weight_cols"] = [None, "num_channels", "capacity", "fee_base_msat", "fee_rate_milli_msat"]
DEFAULTS["snapshot_ids"] = list(range(9))

In [ ]:
DEFAULTS["experiment_id"] = "trial"
DEFAULTS["snapshot_id"] = 0
DEFAULTS["num_transactions"] = 5000
DEFAULTS["amount_sat"] = 50000
DEFAULTS["alpha"] = None
DEFAULTS["day_interval"] = 7
DEFAULTS["drop_disabled"] = True

In [ ]:
DEFAULTS["sim_dir"] = "2019-09-03_15:28:05_50000sat_k5000_aNone_dropFalse"

# Setting parameters for notebooks

In [ ]:
PARAMETERS = {}

In [ ]:
for nb_name in cg.pythonitem_names:
    PARAMETERS[nb_name] = []
print(PARAMETERS)

#### Set your parameters for each notebooks below: 

##### You can specify the ending of the names of the generated notebook clones, if you add an "\_\_ ending \_\_" parameter. (Otherwise they will end with \_CLONE\_1, \_CLONE\_2, etc.)
##### WARNING: if the name of the generated notebook clone is not an identifier, then luigi will NOT be able to run it.

## Centrality computation

for w in DEFAULTS["weight_cols"]:
    PARAMETERS["CentralityCalculator"].append({"weight":w})

## Transaction simulation

import datetime as dt

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 5#10
alpha = None

#drop_disabled = True
#for amount, num_transactions in [(50000,6500)]:
#    for snap in range(7):
#        for i in range(num_samples):
#            PARAMETERS['RunSimulator'].append({"snapshot_id":snap, "drop_disabled":drop_disabled, "alpha":alpha, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i_a%s_drop%s.%i" % (start_time, amount, num_transactions, alpha, drop_disabled, i)})

for sat in [10000,25000,50000,100000,200000,500000]:
    drop_disabled = True#False
    for amount, num_transactions in [(sat,6000)]:
        for snap in range(7):
            for i in range(num_samples):
                PARAMETERS['RunSimulator'].append({"snapshot_id":snap, "drop_disabled":drop_disabled, "alpha":alpha, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i_a%s_drop%s.%i" % (start_time, amount, num_transactions, alpha, drop_disabled, i)})

for amount in [1000,10000,100000,1000000]:
    PARAMETERS['RunSimulator'].append({""})

## Transaction results

sim_dirs = [
    "2019-09-05_14:35:13_10000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_25000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_50000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_100000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_200000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_500000sat_k6000_aNone_dropTrue",
]

sim_dirs = [
    "2019-09-04_14:12:21_500000sat_k5000_aNone_dropFalse",
    "2019-09-04_14:12:21_200000sat_k5000_aNone_dropFalse",
    "2019-09-02_17:05:18_100000sat_k5000_aNone_dropFalse",
    "2019-09-03_15:28:05_50000sat_k5000_aNone_dropFalse",
    "2019-09-04_14:12:21_25000sat_k5000_aNone_dropFalse",
    "2019-09-04_14:12:21_10000sat_k5000_aNone_dropFalse"
]

In [ ]:
for d in sim_dirs:
    PARAMETERS['SimulationResults'].append({"sim_dir":d})

# Export parameters into pipeline configuration file

In [ ]:
cg.save_params(PARAMETERS, DEFAULTS)

cg.pipeline_cfg